In [ ]:
# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

# Root directory for dataset
dataroot = "C:/Users/Abhishek Mukherjee/Documents/DCGANs/Data"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 32

# Spatial size of training images. All images will be resized to this
#size using a transformer.
image_size = 64

# Creating the transformations
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.



In [ ]:
# Loading the dataset for image corpus
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)



In [ ]:
## Loading the dataset for tensor images
##Commented while using image corpus
#dataset = dset.CIFAR10(root = 'C:/Users/Abhishek Mukherjee/Downloads/Module 3 - GANs/data/cifar-10-batches-py', download = True, transform = transform) # We download the training set in the ./data folder and we apply the previous transformations on each image.
#dataloader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = True, num_workers =workers) # We use dataLoader to get the images of the training set batch by batch.

In [ ]:
#Loses vector
lossG=[]
lossD=[]

# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# Defining the generator
class G(nn.Module):

    def __init__(self): 
        super(G, self).__init__() 
        self.main = nn.Sequential( 
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False), 
            nn.BatchNorm2d(512), 
            nn.ReLU(True), 
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False), 
            nn.BatchNorm2d(256),
            nn.ReLU(True), 
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False), 
            nn.BatchNorm2d(128), 
            nn.ReLU(True), 
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64), 
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False), 
            nn.Tanh() 
        )

    def forward(self, input): 
        output = self.main(input) 
        return output 

# Creating the generator
netG = G() # We create the generator object.
netG.apply(weights_init) # We initialize all the weights of its neural network.

# Defining the discriminator

class D(nn.Module): # We introduce a class to define the discriminator.

    def __init__(self): 
        super(D, self).__init__() 
        self.main = nn.Sequential( 
            nn.Conv2d(3, 64, 4, 2, 1, bias = False), # We start with a convolution.
            nn.LeakyReLU(0.2, inplace = True), # We apply a LeakyReLU.
            nn.Conv2d(64, 128, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(128), # We normalize all the features along the dimension of the batch.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(128, 256, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(256), # We normalize again.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(256, 512, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(512), # We normalize again.
            nn.LeakyReLU(0.2, inplace = True), # We apply another LeakyReLU.
            nn.Conv2d(512, 1, 4, 1, 0, bias = False), # We add another convolution.
            nn.Sigmoid() # We apply a Sigmoid rectification to break the linearity and stay between 0 and 1.
        )

    def forward(self, input): 
        output = self.main(input) 
        return output.view(-1) 

# Creating the discriminator
netD = D() 
netD.apply(weights_init)

# Training the DCGANs

criterion = nn.BCELoss() 
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999)) 
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999)) 

In [ ]:
#Num. of epochs: 
totEpochs=35

for epoch in range(totEpochs): 

    for i, data in enumerate(dataloader, 0): 
        
        #Updating the weights of the neural network of the discriminator
        netD.zero_grad() 
        
        # Training the discriminator with a real image of the dataset
        real, _ = data 
        input = Variable(real) 
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input) 
        errD_real = criterion(output, target) 
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)) 
        fake = netG(noise) 
        errD_fake = criterion(output, target) 

        # Backpropagating the total error
        errD = errD_real + errD_fake 
        errD.backward()
        optimizerD.step() 

        # Updating the weights of the neural network of the generator
        netG.zero_grad() 
        target = Variable(torch.ones(input.size()[0])) 
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward() 
        optimizerG.step() 
        
        #Printing the losses and saving the real images and the generated images of the minibatch every 100 steps
        print(f'current epoch is {epoch}, value of the image number in data-loader {i} \n Discriminator loss is {errD.data.item()}, Generator loss is {errG.data.item()}')
        
        #Appending teh losses to plot loss at the end
        lossG.append(errG.item())
        lossD.append(errD.item())
        
        if i % 100 == 0: # Every 100 steps:
            vutils.save_image(real, '%s/real_samples.png' % "C:/Users/Abhishek Mukherjee/Documents/DCGANs/results", normalize = True) # We save the real images of the minibatch.
            fake = netG(noise) 
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("C:/Users/Abhishek Mukherjee/Documents/DCGANs/results", epoch), normalize = True) # We also save the fake generated images of the minibatch.